<a href="https://colab.research.google.com/github/ryanzambrano/scout-ai/blob/main/model/scout_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler

In [102]:
centerURL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/c.csv'
pfURL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/pf.csv'
pgURL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/pg.csv'
sfURL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/sf.csv'
sgURL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/sg.csv'
nbaStatsURL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/nbaStats.csv'

In [103]:
CenterDF = pd.read_csv(centerURL)

features = CenterDF.drop(['#', 'Player', 'Team'], axis=1)
target = CenterDF['#']

# Normalize the features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

rank_correlations = features.corrwith(target, method='pearson')
positive_rank_correlations = rank_correlations[rank_correlations > 0]
threshold = 0.3  # Set your desired threshold
positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 5  # Set the number of top features
top_N_correlations = rank_correlations.nlargest(top_N)
print(top_N_correlations)

FG%    0.151086
3P%   -0.079483
GP    -0.333568
FT%   -0.372363
3PM   -0.427661
dtype: float64


In [104]:
PowerForwardDF = pd.read_csv(pfURL)
rank_column = "#"
features = PowerForwardDF.columns[1:]

rank_correlations = PowerForwardDF[features].corrwith(PowerForwardDF[rank_column], method='pearson')
positive_rank_correlations = rank_correlations[rank_correlations > 0]
threshold = 0.3  # Set your desired threshold
positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 5  # Set the number of top features
top_N_correlations = rank_correlations.nlargest(top_N)
print(top_N_correlations)

3P%   -0.067441
GP    -0.196214
ORB   -0.358369
FT%   -0.409817
PF    -0.424395
dtype: float64


<ipython-input-104-e5ca67cdcef8>:5: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  rank_correlations = PowerForwardDF[features].corrwith(PowerForwardDF[rank_column], method='pearson')


In [105]:
ShootingGuardDF = pd.read_csv(sgURL)

rank_column = "#"
features = ShootingGuardDF.columns[1:]

rank_correlations = ShootingGuardDF[features].corrwith(ShootingGuardDF[rank_column], method='pearson')
positive_rank_correlations = rank_correlations[rank_correlations > 0]
threshold = 0.3  # Set your desired threshold
positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 5  # Set the number of top features
top_N_correlations = rank_correlations.nlargest(top_N)
print(top_N_correlations)

3P%   -0.078622
ORB   -0.179126
FG%   -0.188434
GP    -0.278553
BPG   -0.292484
dtype: float64


<ipython-input-105-8fffbd87c87f>:6: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  rank_correlations = ShootingGuardDF[features].corrwith(ShootingGuardDF[rank_column], method='pearson')


In [106]:
PointGuardDF = pd.read_csv(pgURL)
rank_column = "#"
features = PointGuardDF.columns[1:]

rank_correlations = PointGuardDF[features].corrwith(PointGuardDF[rank_column], method='pearson')
positive_rank_correlations = rank_correlations[rank_correlations > 0]
threshold = 0.3  # Set your desired threshold
positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 5  # Set the number of top features
top_N_correlations = rank_correlations.nlargest(top_N)
print(top_N_correlations)

GP    -0.008395
ORB   -0.180176
3P%   -0.272761
FG%   -0.341911
FT%   -0.398390
dtype: float64


<ipython-input-106-f24746d9c5ba>:5: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  rank_correlations = PointGuardDF[features].corrwith(PointGuardDF[rank_column], method='pearson')


In [107]:
SmallForwardDF = pd.read_csv(sfURL)
rank_column = "#"
features = SmallForwardDF.columns[1:]

rank_correlations = SmallForwardDF[features].corrwith(SmallForwardDF[rank_column], method='pearson')
positive_rank_correlations = rank_correlations[rank_correlations > 0]
threshold = 0.3  # Set your desired threshold
positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 5  # Set the number of top features
top_N_correlations = rank_correlations.nlargest(top_N)
print(top_N_correlations)

GP    -0.192331
BPG   -0.195964
3P%   -0.206452
ORB   -0.208009
FG%   -0.293963
dtype: float64


<ipython-input-107-bc35ab6b469e>:5: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  rank_correlations = SmallForwardDF[features].corrwith(SmallForwardDF[rank_column], method='pearson')


The linear correlations are negative and close to 0 in many cases, as such it is not reasonable to use a linear model to find the important features in the next section I will try to implement a decision tree to try and output rank, from this I will try to find the sequence of features that lead to higher positions AKA the left side of the tree and see if these features make sense to use to predict ranking.

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# Extract features and target variables
features = CenterDF.drop('#', axis=1)
features = features.drop('Player', axis=1)
features = features.drop('Team', axis=1)
target = CenterDF['#']

# Normalize the features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Get column names for the features
feature_columns = features.columns

num_buckets = 11

# Determine the number of players per bucket (assuming a total of 55 players)
players_per_bucket = 55 // num_buckets

# Create an array to hold the buckets
rank_buckets = np.zeros_like(target)

# Assign ranks to buckets
for i in range(num_buckets):
    start_rank = i * players_per_bucket + 1
    end_rank = min((i + 1) * players_per_bucket, 55)
    rank_buckets[start_rank - 1:end_rank] = i + 1

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.3, random_state=13)
X_train_bucket, X_test_bucket, y_train_bucket, y_test_bucket = train_test_split(features_scaled, rank_buckets, test_size=0.3, random_state=13)

# Create and train the decision tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Get the feature importances
importances = clf.feature_importances_

# Create a dataframe with feature names and importances
feature_importances = pd.DataFrame({'Feature': feature_columns, 'Importance': importances})

# Sort features by importance in descending order
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Print the top 5 most important features
print(feature_importances.head(5))

# Create and train the decision tree classifier for bucket classification
clf_bucket = DecisionTreeClassifier()
clf_bucket.fit(X_train_bucket, y_train_bucket)

# Get the feature importances for bucket classification
importances_bucket = clf_bucket.feature_importances_

# Create a dataframe with feature names and importances for bucket classification
feature_importances_bucket = pd.DataFrame({'Feature': feature_columns, 'Importance': importances_bucket})

# Sort features by importance in descending order for bucket classification
feature_importances_bucket = feature_importances_bucket.sort_values('Importance', ascending=False)

# Print the top 5 most important features for bucket classification
print("Top 5 Features for Bucket Classification:")
print(feature_importances_bucket.head(5))


   Feature  Importance
14     RPG    0.138889
1      MPG    0.111111
19      PF    0.083333
9      FTM    0.055556
18     TOV    0.055556
Top 5 Features for Bucket Classification:
   Feature  Importance
2      PPG    0.706349
4      FGA    0.191511
3      FGM    0.102139
0       GP    0.000000
12     ORB    0.000000


In [109]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

# Assuming you have already loaded your dataset into 'CenterDF'

# Extract features and target variables
features = CenterDF.drop(['#', 'Player', 'Team'], axis=1)
target = CenterDF['#']

# Normalize the features
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

# Encode target variable
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.3, random_state=42)


# Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_feature_importances = pd.DataFrame({'Feature': features.columns, 'Importance': rf_clf.feature_importances_})
rf_top_features = rf_feature_importances.sort_values(by='Importance', ascending=False).head(5)

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
gb_feature_importances = pd.DataFrame({'Feature': features.columns, 'Importance': gb_clf.feature_importances_})
gb_top_features = gb_feature_importances.sort_values(by='Importance', ascending=False).head(5)

# Permutation Importance
perm_importance = permutation_importance(gb_clf, X_test, y_test)
perm_sorted_idx = perm_importance.importances_mean.argsort()[::-1][:5]
perm_top_features = pd.DataFrame({'Feature': features.columns[perm_sorted_idx], 'Importance': perm_importance.importances_mean[perm_sorted_idx]})

# Lasso Regression
lasso_cv = LassoCV(cv=5, random_state=42)
lasso_cv.fit(X_train, y_train)
lasso_top_features = pd.DataFrame({'Feature': features.columns, 'Importance': lasso_cv.coef_})
lasso_top_features = lasso_top_features[lasso_top_features['Importance'] != 0].sort_values(by='Importance', ascending=False).head(5)

# Recursive Feature Elimination with Gradient Boosting
rfe_gb = RFE(estimator=gb_clf, n_features_to_select=5, step=1)
rfe_gb.fit(X_train, y_train)
rfe_gb_top_features = pd.DataFrame({'Feature': features.columns, 'Selected': rfe_gb.support_})
rfe_gb_top_features = rfe_gb_top_features[rfe_gb_top_features['Selected']].drop('Selected', axis=1)



# Print top features for each method
print("Top Features from Random Forest Classifier:")
print(rf_top_features)
print("\nTop Features from Gradient Boosting Classifier:")
print(gb_top_features)
print("\nTop Features from Permutation Importance:")
print(perm_top_features)
print("\nTop Features from Lasso Regression:")
print(lasso_top_features)
print("\nTop Features from Recursive Feature Elimination with Gradient Boosting:")
print(rfe_gb_top_features)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.118e+00, tolerance: 8.127e-01
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 

In [110]:
nbaStats_df = pd.read_csv(nbaStatsURL)
nbaStats_df.fillna(0, inplace=True)
print(nbaStats_df)

       #            Player   Pos  Age   Tm   G  GS    MP   FG   FGA  ...  DRB  \
0      1  Precious Achiuwa  C-PF   24  TOT  45   6  19.9  3.1   6.5  ...  3.6   
1      1  Precious Achiuwa     C   24  TOR  25   0  17.5  3.1   6.8  ...  3.4   
2      1  Precious Achiuwa    PF   24  NYK  20   6  22.9  3.1   6.1  ...  3.9   
3      2       Bam Adebayo     C   26  MIA  42  42  34.5  7.6  15.0  ...  8.3   
4      3      Ochai Agbaji    SG   23  UTA  51  10  19.7  2.1   4.9  ...  1.8   
..   ...               ...   ...  ...  ...  ..  ..   ...  ...   ...  ...  ...   
576  529     Dylan Windler    SF   27  LAL   3   0   3.0  1.0   1.7  ...  0.7   
577  530     James Wiseman     C   22  DET  32   0  14.0  2.5   3.7  ...  2.5   
578  531    Christian Wood    PF   28  LAL  47   1  17.8  2.4   5.3  ...  4.4   
579  532      Delon Wright    PG   31  WAS  29   0  15.1  1.6   4.0  ...  1.5   
580  533    Thaddeus Young    PF   35  TOR  23   6  15.2  2.3   3.8  ...  2.0   

      TRB  AST  STL  BLK  T

In [111]:
print(CenterDF)

     #                Player Team  GP   MPG   PPG   FGM   FGA    FG%  3PM  \
0    1           Joel Embiid  PHI  34  34.0  35.3  11.8  22.2  0.533  1.2   
1    2          Nikola Jokic  DEN  50  33.9  26.3  10.3  17.6  0.584  1.1   
2    3         Anthony Davis  LAL  49  36.0  24.8   9.5  17.2  0.553  0.3   
3    4        Alperen Sengun  HOU  51  32.4  21.3   8.5  15.7  0.541  0.5   
4    5     Victor Wembanyama  SAS  46  28.4  20.3   7.5  16.1  0.464  1.5   
5    6    Kristaps Porzingis  BOS  38  29.8  20.2   6.8  12.9  0.528  1.8   
6    7           Bam Adebayo  MIA  42  34.5  20.2   7.6  15.0  0.509  0.0   
7    8      Domantas Sabonis  SAC  50  35.6  19.9   8.0  12.9  0.620  0.5   
8    9        Nikola Vucevic  CHI  47  34.2  17.3   7.4  15.6  0.475  1.1   
9   10          Myles Turner  IND  50  27.3  17.0   6.2  12.0  0.515  1.4   
10  11         Chet Holmgren  OKC  51  30.2  16.9   6.4  12.0  0.535  1.7   
11  12         Jarrett Allen  CLE  45  30.5  15.4   6.3   9.8  0.641  0.0   

In [112]:

# Extract the unique names of players from both DataFrames
nba_players = set(nbaStats_df['Player'])
center_players = set(CenterDF['Player'])

# Find the intersection of player names between the two DataFrames
common_players = nba_players.intersection(center_players)

# Create a new DataFrame to store the merged data
new_table = pd.DataFrame(columns=['Player', 'Rank'] + list(CenterDF.columns[1:]))

# Iterate over the common player names and retrieve their rank from nbaStats_df and stats from CenterDF
for player in common_players:
    rank = nbaStats_df.loc[nbaStats_df['Player'] == player, '#'].values[0]
    player_stats = CenterDF.loc[CenterDF['Player'] == player].values.tolist()[0][1:]
    new_row = [player, rank] + player_stats
    new_table.loc[len(new_table)] = new_row

sorted_table = new_table.sort_values(by='Rank')
# Display the new table
print(sorted_table)


                  Player  Rank                Player Team  GP   MPG   PPG  \
16           Bam Adebayo     2           Bam Adebayo  MIA  42  34.5  20.2   
9           Santi Aldama     4          Santi Aldama  MEM  40  24.4  10.7   
13         Jarrett Allen     7         Jarrett Allen  CLE  45  30.5  15.4   
28          Goga Bitadze    42          Goga Bitadze  ORL  43  19.0   6.2   
6          Chris Boucher    53         Chris Boucher  TOR  45  14.0   6.2   
18          Clint Capela    82          Clint Capela  ATL  47  25.7  11.5   
21          John Collins    95          John Collins  UTA  49  27.8  14.4   
11          Zach Collins    96          Zach Collins  SAS  40  24.8  12.0   
5          Anthony Davis   108         Anthony Davis  LAL  49  36.0  24.8   
40        Andre Drummond   123        Andre Drummond  CHI  52  16.5   8.1   
12           Joel Embiid   132           Joel Embiid  PHI  34  34.0  35.3   
30          Drew Eubanks   133          Drew Eubanks  PHX  46  16.2   5.8   

Now that I know i have garbage data, I want to use rankings from 2k that has an entire team dedicated to ranking these players. I will try to join the 2k rankings with the stats on the players and from there use multiple models to determine feature weights

In [113]:
twokC_URL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/2kC.csv'

twokPF_URL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/2kPF.csv'

twokSG_URL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/2kSG.csv'

twokSF_URL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/2kSF.csv'

twokPG_URL = 'https://raw.githubusercontent.com/ryanzambrano/scout-ai/main/model/2kPG.csv'


In [114]:
# prompt: generate five dataframes from the URLs in the block above, append 2k to the name of the variables and then print them all
twokC_df = pd.read_csv(twokC_URL, index_col=False)
twokPF_df = pd.read_csv(twokPF_URL)
twokSG_df = pd.read_csv(twokSG_URL)
twokSF_df = pd.read_csv(twokSF_URL)
twokPG_df = pd.read_csv(twokPG_URL)

print(twokC_df)



      #                                             Player  OVR  3PT  DNK
0     1  Joel Embiid NBA All-Star 34 Badges Joel Embiid...   98   80   80
1     2  Nikola Jokic NBA All-Star 43 Badges Nikola Jok...   98   80   75
2     3  Anthony Davis NBA All-Star 31 Badges Anthony D...   94   70   88
3     4  Victor Wembanyama 11 Badges Victor Wembanyama ...   88   74   75
4     5  Bam Adebayo NBA All-Star 32 Badges Bam Adebayo...   88   65   80
..  ...                                                ...  ...  ...  ...
95   96  Boban Marjanovic 2 Badges Boban Marjanovic C |...   72   63   60
96   97  Kenneth Lofton Jr. 3 Badges Kenneth Lofton Jr....   72   70   75
97   98  Willie Cauley-Stein 3 Badges Willie Cauley-Ste...   72   49   70
98   99  Tony Bradley 7 Badges Tony Bradley C | 6'10" |...   72   26   60
99  100  Chris Silva 0 Badges Chris Silva C / PF | 6'8"...   72   72   65

[100 rows x 5 columns]


In [115]:
# prompt: print the column names for all five dataframes above

print(twokC_df.columns.tolist())
print(twokPF_df.columns.tolist())
print(twokSG_df.columns.tolist())
print(twokSF_df.columns.tolist())
print(twokPG_df.columns.tolist())


['#', 'Player', 'OVR', '3PT', 'DNK']
['#', 'Player', 'OVR', '3PT', 'DNK']
['#', 'Player', 'OVR', '3PT', 'DNK']
['#', 'Player', 'OVR', '3PT', 'DNK']
['#', 'Player', 'OVR', '3PT', 'DNK', 'Unnamed: 5', 'Unnamed: 6']


In [116]:
twokPG_df = twokPG_df.drop(columns = ['Unnamed: 5', 'Unnamed: 6'])

In [117]:
twokC_df['Player'] = twokC_df['Player'].apply(lambda x: ' '.join(x.split(' ', 2)[:2]))
twokPF_df['Player'] = twokPF_df['Player'].apply(lambda x: ' '.join(x.split(' ', 2)[:2]))
twokSG_df['Player'] = twokSG_df['Player'].apply(lambda x: ' '.join(x.split(' ', 2)[:2]))
twokSF_df['Player'] = twokSF_df['Player'].apply(lambda x: ' '.join(x.split(' ', 2)[:2]))
twokPG_df['Player'] = twokPG_df['Player'].apply(lambda x: ' '.join(x.split(' ', 2)[:2]))
print(twokC_df)

      #               Player  OVR  3PT  DNK
0     1          Joel Embiid   98   80   80
1     2         Nikola Jokic   98   80   75
2     3        Anthony Davis   94   70   88
3     4    Victor Wembanyama   88   74   75
4     5          Bam Adebayo   88   65   80
..  ...                  ...  ...  ...  ...
95   96     Boban Marjanovic   72   63   60
96   97       Kenneth Lofton   72   70   75
97   98  Willie Cauley-Stein   72   49   70
98   99         Tony Bradley   72   26   60
99  100          Chris Silva   72   72   65

[100 rows x 5 columns]


In [149]:
# prompt: join the two twokC_df with the CenterDF, the twokC_df should primary key should be the player names from the twokC_df

merged_Cdf = pd.merge(twokC_df, CenterDF, on='Player', how='inner')
merged_Cdf = merged_Cdf.drop(columns=['Player', 'OVR', '3PT', 'DNK', '#_y', 'Team'])

features = merged_Cdf.drop(columns=['#_x'])
target = merged_Cdf['#_x']
# Normalize the features
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)
features_df = pd.DataFrame(features_normalized, columns=features.columns)

rank_correlations = features.corrwith(target, method='pearson')
#threshold = 0.3  # Set your desired threshold
#positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 23  # Set the number of top features
top_N_correlations_C = rank_correlations.nsmallest(top_N)
print(top_N_correlations_C)

MPG   -0.913178
DRB   -0.897894
RPG   -0.873599
FGM   -0.862442
PPG   -0.830803
FGA   -0.820367
TOV   -0.712614
FTA   -0.692660
FTM   -0.657585
SPG   -0.657170
PF    -0.643554
ORB   -0.611623
APG   -0.600676
BPG   -0.547676
GP    -0.480833
3PA   -0.356631
3PM   -0.334751
FT%   -0.321952
3P%    0.010239
FG%    0.048578
dtype: float64


In [156]:
print(features_df)

        GP       MPG       PPG       FGM       FGA      FG%       3PM  \
0   0.4375  1.000000  1.000000  1.000000  1.000000  0.74375  0.812500   
1   0.5625  0.833333  0.789809  0.747573  0.788177  0.58750  1.000000   
2   0.1875  0.775194  0.601911  0.631068  0.610837  0.78750  0.625000   
3   0.6875  0.906977  0.694268  0.582524  0.679803  0.35625  0.583333   
4   0.7500  0.926357  0.764331  0.805825  0.817734  0.70000  0.541667   
5   0.6250  0.961240  0.767516  0.689320  0.793103  0.39375  0.687500   
6   0.1250  0.771318  0.563694  0.631068  0.645320  0.62500  0.500000   
7   0.5000  0.875969  0.452229  0.359223  0.384236  0.50000  0.562500   
8   0.0625  0.779070  0.512739  0.543689  0.551724  0.63750  0.708333   
9   0.2500  0.872093  0.611465  0.679612  0.743842  0.48750  0.354167   
10  0.6250  0.961240  0.423567  0.398058  0.487685  0.24375  0.583333   
11  0.1250  0.651163  0.398089  0.407767  0.448276  0.43750  0.416667   
12  0.8125  0.422481  0.257962  0.310680  0.305419 

In [150]:
# prompt: tweak the code above so that it correlates twokPF_df and PowerForwardDF

merged_PFdf = pd.merge(twokPF_df, PowerForwardDF, on='Player', how='inner')
merged_PFdf = merged_PFdf.drop(columns=['Player', 'OVR', '3PT', 'DNK', '#_y', 'Team'])

features = merged_PFdf.drop(columns=['#_x'])
target = merged_PFdf['#_x']
# Normalize the features
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)
features_df = pd.DataFrame(features_normalized, columns=features.columns)

rank_correlations = features.corrwith(target, method='pearson')
#threshold = 0.3  # Set your desired threshold
#positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 23  # Set the number of top features
top_N_correlations_PF = rank_correlations.nsmallest(top_N)
print(top_N_correlations_PF)


FGM   -0.880608
MPG   -0.874855
PPG   -0.872140
FGA   -0.856958
TOV   -0.804991
FTM   -0.800538
FTA   -0.776557
APG   -0.748916
FG%   -0.709752
RPG   -0.705970
DRB   -0.683017
PF    -0.593583
SPG   -0.564934
ORB   -0.496339
GP    -0.420148
BPG   -0.405728
3PA   -0.256632
3PM   -0.231898
FT%   -0.178418
3P%   -0.049081
dtype: float64


In [151]:
# prompt: tweak the code above so that it correlates twokSG_df and ShootingGuardDF

merged_SGdf = pd.merge(twokSG_df, ShootingGuardDF, on='Player', how='inner')
merged_SGdf = merged_SGdf.drop(columns=['Player', 'OVR', '3PT', 'DNK', '#_y', 'Team'])

features = merged_SGdf.drop(columns=['#_x'])
target = merged_SGdf['#_x']
# Normalize the features
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)
features_df = pd.DataFrame(features_normalized, columns=features.columns)

rank_correlations = features.corrwith(target, method='pearson')
#threshold = 0.3  # Set your desired threshold
#positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 23  # Set the number of top features
top_N_correlations_SG = rank_correlations.nsmallest(top_N)
print(top_N_correlations_SG)


PPG   -0.889287
FGM   -0.875528
FGA   -0.864955
APG   -0.787698
FTM   -0.784775
FTA   -0.779293
MPG   -0.775710
TOV   -0.764127
3PA   -0.758201
DRB   -0.712366
RPG   -0.712079
3PM   -0.685848
SPG   -0.559554
ORB   -0.425797
FG%   -0.409400
PF    -0.367934
BPG   -0.325789
FT%   -0.300605
GP    -0.112561
3P%   -0.078712
dtype: float64


In [152]:
# prompt: tweak the code above so that it correlates twokSF_df and SmallForwardDF

merged_SFdf = pd.merge(twokSF_df, SmallForwardDF, on='Player', how='inner')
merged_SFdf = merged_SFdf.drop(columns=['Player', 'OVR', '3PT', 'DNK', '#_y', 'Team'])

features = merged_SFdf.drop(columns=['#_x'])
target = merged_SFdf['#_x']
# Normalize the features
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)
features_df = pd.DataFrame(features_normalized, columns=features.columns)

rank_correlations = features.corrwith(target, method='pearson')
#threshold = 0.3  # Set your desired threshold
#positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 23  # Set the number of top features
top_N_correlations_SF = rank_correlations.nsmallest(top_N)
print(top_N_correlations_SF)


FGM   -0.884041
FGA   -0.879003
PPG   -0.878281
MPG   -0.870286
TOV   -0.856269
RPG   -0.741996
APG   -0.729655
FTM   -0.723060
FTA   -0.719796
DRB   -0.710230
PF    -0.548747
SPG   -0.518699
ORB   -0.469617
FG%   -0.372310
3PA   -0.341241
FT%   -0.329629
3PM   -0.291282
BPG   -0.278433
GP     0.006407
3P%    0.034640
dtype: float64


In [153]:
# prompt: tweak the code above so that it correlates twokPG_df and PointGuardDF

merged_PGdf = pd.merge(twokPG_df, PointGuardDF, on='Player', how='inner')
merged_PGdf = merged_PGdf.drop(columns=['Player', 'OVR', '3PT', 'DNK', '#_y', 'Team'])

features = merged_PGdf.drop(columns=['#_x'])
target = merged_PGdf['#_x']
# Normalize the features
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)
features_df = pd.DataFrame(features_normalized, columns=features.columns)

rank_correlations = features.corrwith(target, method='pearson')
#threshold = 0.3  # Set your desired threshold
#positive_rank_correlations = rank_correlations[rank_correlations > threshold]
top_N = 23  # Set the number of top features
top_N_correlations_PG = rank_correlations.nsmallest(top_N)
print(top_N_correlations_PG)


MPG   -0.908226
FGM   -0.887656
PPG   -0.884572
FGA   -0.884141
APG   -0.821296
SPG   -0.793938
3PA   -0.793662
3PM   -0.782228
FTA   -0.781092
FTM   -0.778830
TOV   -0.769541
DRB   -0.746767
RPG   -0.732244
BPG   -0.541242
PF    -0.458776
FG%   -0.454682
ORB   -0.320361
FT%   -0.298991
3P%   -0.184766
GP    -0.047399
dtype: float64
